#### Importações

In [97]:
import pandas as pd
import matplotlib.pyplot as plt

# Configurações para exibição do DataFrame
pd.set_option('display.max_columns', None)  # Mostrar todas as colunas
pd.set_option('display.max_rows', None)     # Mostrar todas as linhas
pd.set_option('display.max_colwidth', None) # Mostrar todo o conteúdo das células

#### Importando os dataframes que serão analisados

In [98]:
df_original = pd.read_csv("../dados/nova_plataforma.csv")
df_gan = pd.read_csv("../dados/registros_gan.csv")

##### Removendo as colunas de df_original que não estão na df_gan e df_regessao

In [99]:
colunas_para_remover = ['platform', 'recommended (1 partial; 2 complete)']
df_original.drop(colunas_para_remover, axis=1, inplace=True)

#### Analisando os registros gerados pela gan

##### Concatenado as estatisticas obtidas do df_original e df_gan

In [100]:
# Obtendo estatísticas para df_original e df_gan
describe_original = df_original.describe().transpose()
describe_gan = df_gan.describe().transpose()

# Removendo a coluna count, pois nesse caso apenas diz a quantidade de registros. 
# Portanto, não precisamos dela para a comparação
describe_original.drop(columns='count', inplace=True)
describe_gan.drop(columns='count', inplace=True)

# Adicionando sufixos aos nomes das colunas para distinguir entre os dois DataFrames
describe_original = describe_original.add_suffix('_original')
describe_gan = describe_gan.add_suffix('_gan')

# Criando DataFrame vazio para armazenar estatísticas intercaladas
original_x_gan = pd.DataFrame()

# Intercalando as colunas dos DataFrames originais e sintéticos
for coluna_original, coluna_gan in zip(describe_original.columns, describe_gan.columns):
    original_x_gan[coluna_original] = describe_original[coluna_original]
    original_x_gan[coluna_gan] = describe_gan[coluna_gan]


In [101]:
original_x_gan.head(25)

,mean_original,mean_gan,std_original,std_gan,min_original,min_gan,25%_original,25%_gan,50%_original,50%_gan,75%_original,75%_gan,max_original,max_gan
water_depth (m),124.428571,124.240000,22.307377,18.950248,99.000,99.000000,107.500,1.060000e+02,1.160000e+02,1.220000e+02,142.500000,1.420000e+02,1.560000e+02,1.550000e+02
weight (t),18815.714286,18958.760100,5472.264453,4504.916363,10054.000,10127.000000,16530.500,1.522125e+04,1.858400e+04,1.913100e+04,21182.000000,2.276525e+04,2.764700e+04,2.757900e+04
installation_date,38.857143,36.323800,5.639993,4.192483,31.000,31.000000,34.500,3.300000e+01,4.100000e+01,3.500000e+01,42.500000,3.900000e+01,4.600000e+01,4.500000e+01
type_of_production (1 oil and gas; 2 oil; 3 gas),1.571429,1.360000,0.786796,0.678119,1.000,1.000000,1.000,1.000000e+00,1.000000e+00,1.000000e+00,2.000000,1.000000e+00,3.000000e+00,3.000000e+00
number_of_legs,7.428571,6.760400,1.511858,1.849901,4.000,4.000000,8.000,4.000000e+00,8.000000e+00,8.000000e+00,8.000000,8.000000e+00,8.000000e+00,8.000000e+00
number_of_piles,17.142857,13.776000,10.106575,9.640151,5.000,5.000000,8.500,5.000000e+00,2.000000e+01,9.000000e+00,23.000000,2.300000e+01,3.200000e+01,3.100000e+01
height_of_jacket_or_sub-structure (m),137.500000,136.810202,18.993420,17.609062,114.000,114.001000,122.750,1.192498e+02,1.360000e+02,1.346570e+02,150.500000,1.538428e+02,1.660000e+02,1.659660e+02
distance_to_coast (km),218.571429,229.414400,66.271807,48.529366,120.000,120.000000,180.000,1.900000e+02,2.400000e+02,2.450000e+02,264.000000,2.740000e+02,2.820000e+02,2.810000e+02
risk_to_other_users-complete,0.428571,0.553200,0.534522,0.497187,0.000,0.000000,0.000,0.000000e+00,0.000000e+00,1.000000e+00,1.000000,1.000000e+00,1.000000e+00,1.000000e+00
risk_to_other_users-partial,0.000007,0.000005,0.000009,0.000007,0.000,0.000000,0.000,3.849592e-08,7.800000e-08,9.148168e-07,0.000012,8.380472e-06,2.300000e-05,2.297812e-05


#### Removendo registros que contém células que ultrapassam o intervalo da coluna (min e max)

In [102]:
def ultrapassou_intervalo(registro, coluna, df_original):
    if registro > df_original[coluna].max() or registro < df_original[coluna].min():
        return True
    else:
        return False

In [103]:
def remover_registros_fora_do_intervalo(df_gan, df_original):
    indices_para_remover = []

    for indice, linha in df_gan.iterrows():
        for coluna in df_gan.columns:
            if ultrapassou_intervalo(linha[coluna], coluna, df_original):
                indices_para_remover.append(indice)
                break  # Se um valor exceder o intervalo, já podemos excluir o registro inteiro
    df_gan_limpo = df_gan.drop(indices_para_remover)
    print(len(indices_para_remover))
    return df_gan_limpo

In [104]:
# df_gan_limpo = remover_registros_fora_do_intervalo(df_gan, df_original)

In [105]:
# df_gan_limpo.shape

#### Removendo outliers com base na amplitude interquartil

In [106]:
def remover_outliers_iqr(df_gan):
    indices_para_remover = []

    for indice, linha in df_gan.iterrows():
        for coluna in df_gan.columns:
            q1 = df_gan[coluna].quantile(0.25)
            q3 = df_gan[coluna].quantile(0.75)
            
            iqr = q3 - q1 # Amplitude interquantil
            limite_superior = q3 + 1.5 * iqr
            limite_inferior = q1 - 1.5 * iqr

            if linha[coluna] < limite_inferior or linha[coluna] > limite_superior:
                indices_para_remover.append(indice)
                break  # Se um valor exceder o intervalo, já podemos excluir o registro inteiro

    df_gan_limpo = df_gan.drop(indices_para_remover)
    print(len(indices_para_remover))
    return df_gan_limpo

In [107]:
#df_gan_limpo = remover_outliers_iqr(df_gan)

In [108]:
# df_gan.shape, df_gan_limpo.shape

In [109]:
#df_gan_limpo.duplicated().sum()

#### Comparando novamente as bases de dados

In [110]:
# Obtendo estatísticas para df_original e df_gan
describe_original = df_original.describe().transpose()
describe_gan = df_gan.describe().transpose()

# Removendo a coluna count, pois nesse caso apenas diz a quantidade de registros. 
# Portanto, não precisamos dela para a comparação
describe_original.drop(columns='count', inplace=True)
describe_gan.drop(columns='count', inplace=True)

# Adicionando sufixos aos nomes das colunas para distinguir entre os dois DataFrames
describe_original = describe_original.add_suffix('_original')
describe_gan = describe_gan.add_suffix('_gan')

# Criando DataFrame vazio para armazenar estatísticas intercaladas
original_x_gan = pd.DataFrame()

# Intercalando as colunas dos DataFrames originais e sintéticos
for coluna_original, coluna_gan in zip(describe_original.columns, describe_gan.columns):
    original_x_gan[coluna_original] = describe_original[coluna_original]
    original_x_gan[coluna_gan] = describe_gan[coluna_gan]


In [111]:
original_x_gan.head(26)

,mean_original,mean_gan,std_original,std_gan,min_original,min_gan,25%_original,25%_gan,50%_original,50%_gan,75%_original,75%_gan,max_original,max_gan
water_depth (m),124.428571,124.240000,22.307377,18.950248,99.000,99.000000,107.500,1.060000e+02,1.160000e+02,1.220000e+02,142.500000,1.420000e+02,1.560000e+02,1.550000e+02
weight (t),18815.714286,18958.760100,5472.264453,4504.916363,10054.000,10127.000000,16530.500,1.522125e+04,1.858400e+04,1.913100e+04,21182.000000,2.276525e+04,2.764700e+04,2.757900e+04
installation_date,38.857143,36.323800,5.639993,4.192483,31.000,31.000000,34.500,3.300000e+01,4.100000e+01,3.500000e+01,42.500000,3.900000e+01,4.600000e+01,4.500000e+01
type_of_production (1 oil and gas; 2 oil; 3 gas),1.571429,1.360000,0.786796,0.678119,1.000,1.000000,1.000,1.000000e+00,1.000000e+00,1.000000e+00,2.000000,1.000000e+00,3.000000e+00,3.000000e+00
number_of_legs,7.428571,6.760400,1.511858,1.849901,4.000,4.000000,8.000,4.000000e+00,8.000000e+00,8.000000e+00,8.000000,8.000000e+00,8.000000e+00,8.000000e+00
number_of_piles,17.142857,13.776000,10.106575,9.640151,5.000,5.000000,8.500,5.000000e+00,2.000000e+01,9.000000e+00,23.000000,2.300000e+01,3.200000e+01,3.100000e+01
height_of_jacket_or_sub-structure (m),137.500000,136.810202,18.993420,17.609062,114.000,114.001000,122.750,1.192498e+02,1.360000e+02,1.346570e+02,150.500000,1.538428e+02,1.660000e+02,1.659660e+02
distance_to_coast (km),218.571429,229.414400,66.271807,48.529366,120.000,120.000000,180.000,1.900000e+02,2.400000e+02,2.450000e+02,264.000000,2.740000e+02,2.820000e+02,2.810000e+02
risk_to_other_users-complete,0.428571,0.553200,0.534522,0.497187,0.000,0.000000,0.000,0.000000e+00,0.000000e+00,1.000000e+00,1.000000,1.000000e+00,1.000000e+00,1.000000e+00
risk_to_other_users-partial,0.000007,0.000005,0.000009,0.000007,0.000,0.000000,0.000,3.849592e-08,7.800000e-08,9.148168e-07,0.000012,8.380472e-06,2.300000e-05,2.297812e-05


In [112]:
describe_diferenca = (df_original.describe() - df_gan.describe()).transpose()
describe_diferenca.drop(columns='count', inplace=True)
describe_diferenca

,mean,std,min,25%,50%,75%,max
water_depth (m),0.188571,3.357128,0.000000,1.500000e+00,-6.000000e+00,0.500000,1.000000e+00
weight (t),-143.045814,967.348089,-73.000000,1.309250e+03,-5.470000e+02,-1583.250000,6.800000e+01
installation_date,2.533343,1.447511,0.000000,1.500000e+00,6.000000e+00,3.500000,1.000000e+00
type_of_production (1 oil and gas; 2 oil; 3 gas),0.211429,0.108676,0.000000,0.000000e+00,0.000000e+00,1.000000,0.000000e+00
number_of_legs,0.668171,-0.338043,0.000000,4.000000e+00,0.000000e+00,0.000000,0.000000e+00
number_of_piles,3.366857,0.466424,0.000000,3.500000e+00,1.100000e+01,0.000000,1.000000e+00
height_of_jacket_or_sub-structure (m),0.689798,1.384358,-0.001000,3.500250e+00,1.343000e+00,-3.342750,3.400000e-02
distance_to_coast (km),-10.842971,17.742441,0.000000,-1.000000e+01,-5.000000e+00,-10.000000,1.000000e+00
risk_to_other_users-complete,-0.124629,0.037336,0.000000,0.000000e+00,-1.000000e+00,0.000000,0.000000e+00
risk_to_other_users-partial,0.000002,0.000002,0.000000,-3.849592e-08,-8.368168e-07,0.000004,2.187830e-08


#### Analisando a semelhança dos dataframe com base no KSComplement e CorrelationSimilarity

In [113]:
from sdmetrics.reports.single_table import QualityReport
from sdv.metadata import SingleTableMetadata

In [114]:
df_original.head()

,water_depth (m),weight (t),installation_date,type_of_production (1 oil and gas; 2 oil; 3 gas),number_of_legs,number_of_piles,height_of_jacket_or_sub-structure (m),distance_to_coast (km),risk_to_other_users-complete,risk_to_other_users-partial,risk_to_personnel-complete,risk_to_personnel-partial,energy_consumption-complete (GJ),energy_consumption-partial (GJ),emissions-complete (t),emissions-partial (t),impacts_of_option-complete,impacts_of_option-partial,technical_feasibility_or_challenge-complete,technical_feasibility_or_challenge-partial,commercial_impact_on_fisheries-complete,commercial_impact_on_fisheries-partial,wider_community_impact-complete,wider_community_impact-partial,total_removal_cost-complete,total_removal_cost-partial
0,99,22000,36,1,8,8,114.0,269,1,0.000000e+00,0.290,0.12,1110100,817000,87000,58500,0.00,1.0,0.00,1.00,0.79,0.78,0.5,0.5,1.00,0.56
1,112,20364,41,1,8,9,123.0,259,1,0.000000e+00,0.320,0.16,1180500,895500,92000,64000,0.00,1.0,0.00,1.00,0.79,0.78,0.5,0.5,1.00,0.56
2,141,15561,46,2,8,26,147.0,120,0,2.300000e-05,0.025,0.01,297654,530148,24277,31064,0.66,1.0,0.25,1.00,1.00,0.94,1.0,1.0,0.53,1.00
3,156,27647,44,2,8,32,166.0,240,0,1.500000e-05,0.040,0.02,487750,570818,40416,45266,1.00,1.0,0.50,1.00,1.00,0.66,1.0,1.0,1.00,0.57
4,103,18584,33,1,8,20,122.5,230,0,7.800000e-08,0.090,0.06,733082,511765,59588,41170,0.00,1.0,0.39,0.15,0.00,0.00,0.0,0.0,1.00,0.73


In [115]:
df_gan.head()

,water_depth (m),weight (t),installation_date,type_of_production (1 oil and gas; 2 oil; 3 gas),number_of_legs,number_of_piles,height_of_jacket_or_sub-structure (m),distance_to_coast (km),risk_to_other_users-complete,risk_to_other_users-partial,risk_to_personnel-complete,risk_to_personnel-partial,energy_consumption-complete (GJ),energy_consumption-partial (GJ),emissions-complete (t),emissions-partial (t),impacts_of_option-complete,impacts_of_option-partial,technical_feasibility_or_challenge-complete,technical_feasibility_or_challenge-partial,commercial_impact_on_fisheries-complete,commercial_impact_on_fisheries-partial,wider_community_impact-complete,wider_community_impact-partial,total_removal_cost-complete,total_removal_cost-partial
0,106,22111,35,1,8,7,123.033,266,1,1.602288e-07,0.271,0.115,938135,703299,82169,55292,0.004,0.992964,0.008,0.948,0.545,0.589,0.315,0.449,0.999,0.532
1,101,26116,35,1,8,9,114.366,264,1,8.686751e-09,0.303,0.148,1142769,778676,91771,62200,0.000,0.998604,0.025,0.666,0.063,0.163,0.017,0.219,1.000,0.595
2,116,20231,35,1,8,23,134.199,227,0,6.590470e-07,0.108,0.094,753573,561341,78281,50189,0.007,0.916945,0.468,0.594,0.168,0.056,0.042,0.187,0.994,0.570
3,132,19412,38,1,8,9,146.634,239,1,5.273908e-06,0.053,0.051,542365,528817,27399,46774,0.088,0.991737,0.005,0.997,0.978,0.914,0.962,0.694,0.959,0.583
4,131,15140,37,1,4,6,145.660,265,1,1.206552e-06,0.074,0.048,455573,580261,26934,42065,0.170,0.997594,0.001,1.000,0.998,0.935,0.960,0.792,0.994,0.487


In [116]:
# Supondo que A e B são seus DataFrames reais e sintéticos
A = df_original.copy()
B = df_gan.copy()

# # Criar o metadata
# metadata = SingleTableMetadata()
# metadata.detect_from_dataframe(data=A)
# metadata_dict = metadata.to_dict()
metadata_dict = {
    "columns": {
        "water_depth (m)": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "weight (t)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "installation_date": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "type_of_production (1 oil and gas; 2 oil; 3 gas)": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "number_of_legs": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "number_of_piles": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "height_of_jacket_or_sub-structure (m)": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "distance_to_coast (km)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "risk_to_other_users-complete": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "risk_to_other_users-partial": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "risk_to_personnel-complete": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "risk_to_personnel-partial": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "energy_consumption-complete (GJ)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "energy_consumption-partial (GJ)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "emissions-complete (t)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "emissions-partial (t)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "impacts_of_option-complete": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "impacts_of_option-partial": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "technical_feasibility_or_challenge-complete": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "technical_feasibility_or_challenge-partial": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "commercial_impact_on_fisheries-complete": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "commercial_impact_on_fisheries-partial": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "wider_community_impact-complete": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "wider_community_impact-partial": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "total_removal_cost-complete": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "total_removal_cost-partial": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        }
    },
    "primary_key": "water_depth (m)",
    "METADATA_SPEC_VERSION": "SINGLE_TABLE_V1"
}

# Gerar o relatório de qualidade
report = QualityReport()
report.generate(real_data=A, synthetic_data=B, metadata=metadata_dict)

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 26/26 [00:00<00:00, 420.06it/s]|
Column Shapes Score: 69.18%

(2/2) Evaluating Column Pair Trends: |██████████| 325/325 [00:02<00:00, 160.58it/s]|
Column Pair Trends Score: 89.16%

Overall Score (Average): 79.17%



In [117]:
report.get_score()

0.7916710966452729

In [118]:
report.get_properties()

,Property,Score
0,Column Shapes,0.691786
1,Column Pair Trends,0.891556


In [119]:
report.get_details(property_name='Column Shapes')

,Column,Metric,Score
0,water_depth (m),KSComplement,0.847529
1,weight (t),KSComplement,0.838657
2,installation_date,KSComplement,0.630471
3,type_of_production (1 oil and gas; 2 oil; 3 gas),KSComplement,0.816729
4,number_of_legs,KSComplement,0.832957
5,number_of_piles,KSComplement,0.693657
6,height_of_jacket_or_sub-structure (m),KSComplement,0.824057
7,distance_to_coast (km),KSComplement,0.737386
8,risk_to_other_users-complete,KSComplement,0.875371
9,risk_to_other_users-partial,KSComplement,0.573829


In [120]:
report.get_details(property_name='Column Pair Trends')

,Column 1,Column 2,Metric,Score,Real Correlation,Synthetic Correlation
0,water_depth (m),weight (t),CorrelationSimilarity,0.563271,0.242657,-0.630800
1,water_depth (m),installation_date,CorrelationSimilarity,0.964360,0.742407,0.671127
2,water_depth (m),type_of_production (1 oil and gas; 2 oil; 3 gas),CorrelationSimilarity,0.896230,0.828860,0.621320
3,water_depth (m),number_of_legs,CorrelationSimilarity,0.868919,0.166611,-0.095551
4,water_depth (m),number_of_piles,CorrelationSimilarity,0.900846,0.773687,0.575379
5,water_depth (m),height_of_jacket_or_sub-structure (m),CorrelationSimilarity,0.986824,0.980269,0.953916
6,water_depth (m),distance_to_coast (km),CorrelationSimilarity,0.880514,-0.592407,-0.831380
7,water_depth (m),risk_to_other_users-complete,CorrelationSimilarity,0.992926,-0.646966,-0.661114
8,water_depth (m),risk_to_other_users-partial,CorrelationSimilarity,0.991290,0.824868,0.807448
9,water_depth (m),risk_to_personnel-complete,CorrelationSimilarity,0.864526,-0.649726,-0.920674
